In [1]:
import os
import re
import numpy as np
import pandas as pd
from tqdm import tqdm
from tsne import bh_sne
from pathlib import Path
from copy import deepcopy
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from scipy.spatial.distance import euclidean
from sklearn.feature_extraction.text import CountVectorizer
from scipy.cluster.hierarchy import ward, fcluster, dendrogram, linkage

In [2]:
from openpyxl import load_workbook
import xlsxwriter

In [3]:
%%time
INP_DNAME = os.getcwd()
data_fname = '2020 Trade Show Account Matching Worksheet.xlsx'

CPU times: user 8 µs, sys: 1 µs, total: 9 µs
Wall time: 11 µs


In [4]:
# "All Clients" table dataframe
df_all_clients = pd.read_excel(data_fname, sheet_name='All Clients & Prospect Accts')

In [5]:
# Specific table dataframe
df_sheet = pd.read_excel(data_fname, sheet_name='Feb 20 CabanaCapsule')

In [6]:
wb = load_workbook('2020 Trade Show Account Matching Worksheet.xlsx')

In [7]:
list_hupetlink_account = []
for i in range(2, (len(df_all_clients)+2)):
    list_hupetlink_account.append(wb['All Clients & Prospect Accts']['C'+str(i)].value)

In [8]:
df_all_clients['Link'] = list_hupetlink_account

In [9]:
df_all_clients.head(2)

,Parent Account,Account Name,Account ID,Parent Account ID,Account Owner,Account Record Type,Account Currency,Website,Link
0,NaN,Fast Retailing,001i000000UL9l4,NaN,Kate O'Brien,NuORDER Client,USD,NaN,"=HYPERLINK(""https://nuorder.my.salesforce.com/..."
1,Trimera Group Inc,Gottex Swimwear Brands,0013100001lE5vO,0013100001ZJuUR,Jessica Ramsden,NuORDER Client,USD,http://www.gottexmodels.com,"=HYPERLINK(""https://nuorder.my.salesforce.com/..."


In [10]:
df_sheet.head(2)

,Exhibitor,Fuzzy Match Account Name,Account ID,Parent Account ID,Account Owner,Account Record Type,Show,Trade Show ID
0,4our Dreamers,NaN,001i000000phVYa,NaN,NaN,NaN,Capsule,a1I3r000006YHyQEAW
1,8 Paris Rock,NaN,NaN,NaN,NaN,NaN,Capsule,a1I3r000006YHyQEAW


# Clearing and changing data

In [145]:
# Creating customer name lists
list_all_clients = list(df_all_clients['Account Name'])
list_sheet = list(df_sheet['Exhibitor'])

In [146]:
# Function to convert client names to strings
def func_list_name(list_name):
    for i in range(len(list_name)):
        list_name[i] = str(list_name[i])
    return list_name

In [147]:
# List building
list_all_clients = func_list_name(list_all_clients)
list_sheet = func_list_name(list_sheet)

In [148]:
def name_lower(list_name):
    for i in range(len(list_name)):
        list_name[i] = list_name[i].lower()
    return list_name

In [149]:
list_all_clients = name_lower(list_all_clients)
list_sheet = name_lower(list_sheet)

## Шаг 1 Только True Match

In [150]:
# Function for removing punctuation and abbreviations
def clear_list_name(list_name):
    for i in range(len(list_name)):
        list_name[i] = re.sub(r'\W+', ' ', list_name[i])       
        list_name[i] = re.sub(r'\Wco\W|\Winc\W|\Wltd\W|\Wcorp\W|\Wllc\W|\Wplc\W|\Wldc\W|\Wsa\W|\Wbv\W|\Wag\W|\Wslg\W|\Wrsn\W|\Wmfg\W|\Wsrl\W|\Wpty\W|\Wspa\W|\Wsagl\W|\Wdtp\W|\Wthe\W', ' ', list_name[i])
        list_name[i] = re.sub(r'\Wco\b|\Winc\b|\Wltd\b|\Wcorp\b|\Wllc\b|\Wplc\b|\Wldc\b|\Wsa\b|\Wbv\b|\Wag\b|\Wslg\b|\Wrsn\b|\Wmfg\b|\Wsrl\b|\Wpty\b|\Wspa\b|\Wsagl\b|\Wdtp\b|\Wthe\b', ' ', list_name[i])    
        list_name[i] = re.sub(r'\s\w\s', '', list_name[i])
        #list_name[i] = re.sub(r'\W+', '', list_name[i])
    return list_name

In [151]:
# # Function for removing punctuation and abbreviations
# def clear_list_name_hard(list_name):
#     for i in range(len(list_name)):
#         list_name[i] = re.sub(r'\W+', ' ', list_name[i])
        
#         list_name[i] = re.sub(r'\Wusa\W|\Wus\W|\Wau\W|\Wuk\W|\Weu\W', ' ', list_name[i])
#         list_name[i] = re.sub(r'\Wusa\b|\Wus\b|\Wau\b|\Wuk\b|\Weu\b', ' ', list_name[i])
        
#         list_name[i] = re.sub(r'\Wgroup\W|\Winternational\W|\Wcompany\W|\Wamerica\W|\Wkids\W|\Wkidswear\W|\Wshoes\W|\Wmilano\W|\Wwomens\W|\Waustralia\W|\Wbrooklyn\W|\Wparis\W|\Wnew york\W|\Wunited kingdom\W|\Wby\W|\Wlimited\W|\shop\W|\Wtextiles\W|\Wfashion\W|\Wapparel\W|\Wswimwear\W|\Wwear\W|\Wfootwear\W|\Wshoes\W', ' ', list_name[i])
#         list_name[i] = re.sub(r'\Wgroup\b|\Winternational\b|\Wcompany\b|\Wamerica\b|\Wkids\b|\Wkidswear\b|\Wshoes\b|\Wmilano\b|\Wwomens\b|\Waustralia\b|\Wbrooklyn\b|\Wparis\b|\Wnew york\b|\Wunited kingdom\b|\Wby\b|\Wlimited\b|\shop\b|\Wtextiles\b|\Wfashion\b|\Wapparel\b|\Wswimwear\b|\Wwear\b|\Wfootwear\b|\Wshoes\b', ' ', list_name[i])
    
#         list_name[i] = re.sub(r'\s\w\s', '', list_name[i])
#         #list_name[i] = re.sub(r'\W+', '', list_name[i])new york
#     return list_name

In [152]:
def clear_name(name):
    name = re.sub(r'\W+', ' ', name)
    name = re.sub(r'\Wusa\W|\Wus\W|\Wau\W|\Wuk\W|\Weu\W', ' ', name)
    name = re.sub(r'\Wusa\b|\Wus\b|\Wau\b|\Wuk\b|\Weu\b', ' ', name)
        
    name = re.sub(r'\Wgroup\W|\Winternational\W|\Wcompany\W|\Wamerica\W|\Wkids\W|\Wkidswear\W|\Wshoes\W|\Wmilano\W|\Wwomens\W|\Waustralia\W|\Wbrooklyn\W|\Wparis\W|\Wnew york\W|\Wunited kingdom\W|\Wby\W|\Wlimited\W|\shop\W|\Wtextiles\W|\Wfashion\W|\Wapparel\W|\Wswimwear\W|\Wwear\W|\Wfootwear\W|\Wshoes\W', ' ', name)
    name = re.sub(r'\Wgroup\b|\Winternational\b|\Wcompany\b|\Wamerica\b|\Wkids\b|\Wkidswear\b|\Wshoes\b|\Wmilano\b|\Wwomens\b|\Waustralia\b|\Wbrooklyn\b|\Wparis\b|\Wnew york\b|\Wunited kingdom\b|\Wby\b|\Wlimited\b|\shop\b|\Wtextiles\b|\Wfashion\b|\Wapparel\b|\Wswimwear\b|\Wwear\b|\Wfootwear\b|\Wshoes\b', ' ', name)
    
    name = re.sub(r'\s\w\s', '', name)
    return name

In [153]:
# List building
list_all_clients = clear_list_name(list_all_clients)
list_sheet = clear_list_name(list_sheet)

In [154]:
# # Match function
# def find_match(list_sheet_name, list_all_name, sheet_name_match):
#     for i in range(len(list_sheet_name)):
#         if list_sheet_name[i] in list_all_name:
#             sheet_name_match.append(['True Match',list_all_name.index(list_sheet_name[i])])
#         elif list_sheet_name[i] not in list_all_name:
#             list_sheet_name = clear_list_name_hard(list_sheet_name)
#             list_all_name = clear_list_name_hard(list_all_name)
#             if list_sheet_name[i] in list_all_name:
#                 sheet_name_match.append(['Fuzzy Match',list_all_name.index(list_sheet_name[i])])
#             else:                
#                 sheet_name_match.append(0)
#     return sheet_name_match

In [155]:
sheet_name_match = []

In [156]:
for i in tqdm(range(len(list_sheet))):
    for j in range(len(list_all_clients)):
        if list_sheet[i] == list_all_clients[j]:
            sheet_name_match.append(['True Match', list_sheet[i], list_all_clients[j]])
        else:
            if clear_name(list_sheet[i]) == clear_name(list_all_clients[j]):
                sheet_name_match.append(['Fuzzy Match', list_sheet[i], list_all_clients[j]])
            else:
                sheet_name_match.append(' ')

100%|██████████| 244/244 [02:50<00:00,  1.43it/s]


In [160]:
list_sheet[0] == list_all_clients[8977]

True

'4our dreamers'

In [42]:
for i in range(len(list_sheet)):
    if list_sheet[i] in list_all_clients:
        sheet_name_match.append(['True Match', list_all_clients.index(list_sheet[i])])
    elif list_sheet[i] not in list_all_clients:
        list_sheet = clear_list_name_hard(list_sheet)
        list_all_clients = clear_list_name_hard(list_all_clients)
        if list_sheet[i] in list_all_clients:
            sheet_name_match.append(['Fuzzy Match', list_all_clients.index(list_sheet[i])])
        else:                
            sheet_name_match.append(0)

In [43]:
sheet_name_match

[['True Match', 8977],
 0,
 ['True Match', 6257],
 0,
 0,
 ['True Match', 5958],
 0,
 0,
 0,
 0,
 0,
 0,
 ['True Match', 27361],
 0,
 ['True Match', 25077],
 ['True Match', 15615],
 ['True Match', 8307],
 0,
 0,
 ['True Match', 2812],
 0,
 ['True Match', 286],
 0,
 ['True Match', 10983],
 ['True Match', 2564],
 0,
 0,
 0,
 ['True Match', 10132],
 ['True Match', 9957],
 0,
 0,
 0,
 ['True Match', 14255],
 ['True Match', 9646],
 0,
 0,
 0,
 0,
 0,
 ['True Match', 20614],
 0,
 ['True Match', 4901],
 ['True Match', 23151],
 0,
 0,
 ['True Match', 4964],
 0,
 ['True Match', 20298],
 0,
 ['True Match', 16088],
 0,
 ['True Match', 3949],
 0,
 0,
 0,
 0,
 0,
 0,
 ['True Match', 2292],
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 ['True Match', 19926],
 ['True Match', 16935],
 0,
 0,
 0,
 ['True Match', 28170],
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 ['True Match', 5439],
 0,
 ['True Match', 24386],
 0,
 ['True Match', 12086],
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 ['True Match', 2927],
 0,
 ['True 